In [1]:
import csv
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib
%matplotlib inline

In [3]:
data = pd.read_excel("For Data Analysis_2016 Chicago Energy Benchmarking Reporting (2015 Data)_12JUL2016.xlsx")

In [ ]:
def _get_character_pairs(string):
    """
    Returns a defaultdict of adjacent character pair counts.
    Input: string
    Output: defaultdict
    """
    results = defaultdict(lambda: 0)

    for word in string.upper().split():
        for pair in [word[i]+word[i+1] for i in range(len(word)-1)]:
            results[pair] += 1

    return results

def strike_a_match(string1, string2):
    """
    Returns a score between 0.0 and 1.0, with 1 being an exact match.
    Uses strike a match algorithm.
    Input: 2 strings to compare
    Output: float between 0 and 1
    """
    s1_pairs = _get_character_pairs(string1)
    s2_pairs = _get_character_pairs(string2)
    small_dict, large_dict = sorted(
        [s1_pairs, s2_pairs], key=lambda pair_dict: sum(pair_dict.values()))
    intersection_count = 0
    s1_size = sum(s1_pairs.values())
    s2_size = sum(s2_pairs.values())

    for pair, small_pair_count in small_dict.items():
        if pair in large_dict and large_dict[pair] > 0:
            if small_pair_count < large_dict[pair]:
                intersection_count += small_pair_count
            else:
                intersection_count += large_dict[pair]

    return (2.0 * intersection_count) / ((s1_size + s2_size) or 1)

In [ ]:
names = list(data['Unnamed: 1'][4:])
prop_id = list(data['2016 Chicago Energy Benchmarking Reporting (2015 Data)'][4:])
parents = list(data['Unnamed: 2'][4:])
benchmarking_id = list(data['Unnamed: 5'][4:])
address1 = list(data['Unnamed: 6'][4:])

In [ ]:
#for exploring data
data['Unnamed: 197']

In [ ]:
def dedupe(column):
    ''' column is a list
    '''
    duplicates = []
    for i, name in enumerate(column):
        for next_name in column[i+1:]:
            if strike_a_match(str(name), str(next_name)) >= .95:
                duplicates.append((str(name), str(next_name)))
    return list(set(duplicates))

In [ ]:
name_dupes = dedupe(names)
prop_id_dupes = dedupe(prop_id)
parent_dupes = dedupe(parents)
benchmarking_dupes = dedupe(benchmarking_id)
addy_dupes = dedupe(address1)

In [ ]:
sorted(benchmarking_dupes)

In [ ]:
usage_gridpurchase_kwh = data['Unnamed: 35']
usage_gp_kwh = pd.to_numeric(usage_gridpurchase_kwh, errors=('coerce',))
usage_gp_kwh 

In [ ]:
usage_gp_kwh.plot.hist(bins=2)